In [ ]:
pip install transformers

In [ ]:
import pandas as pd
from transformers import pipeline
import torch
# device = torch.device("cpu")
from transformers import GPTJForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer,GPTJForCausalLM
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/MyDrive/rumor_PHEME.csv'
df = pd.read_csv(file_path)

In [ ]:
df.head()

In [ ]:
df.drop(columns=['Sentiment','Emotion','Unnamed: 3'], inplace=True)
df.dropna(inplace=True)
df.head()

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [ ]:
predictions = []
end_sequence='###'

In [ ]:
for index, row in df.iterrows():
    tweet = row['tweet']
    prompt = """
    You are given tweets regarding news events,you need to classify the tweets whether the given tweet is rumor or non-rumor.
    Tweet:Charlie Hebdo became well known for publishing the Muhammed cartoons two years ago
    Answer=non-rumor
    ###
    Tweet:Now 10 dead in a shooting there today RT "BBCDanielS: Charlie Hebdo became well known for publishing the Muhammed cartoons two years ago
    Answer=non-rumor
    ###
    Tweet:BBCDanielS BBCWorld I'm guessing this is being considered terrorism right now or lone wolf?
    Answer:non-rumor
    ###
    Tweet:GabTarquini BBCDanielS BBCWorld Maybe because they shouted "the prophet have been avenged" thus making it relevant to the story?
    Answer:non-rumor
    ###
    Tweet:Once again it shows totall and absolute failure and incompetence of French intelligence and police in preventing terrorism.
    Answer:non-rumor
    ###
    Tweet:kodacohen .BarackObama PAST TIME 2 put a stop to this!Damn what racists will say. Even U r #mikebrown 2 them. World is watching #PBOLegacy
    Answer:non-rumor
    ###
    Tweet:realist_tg JeanieSmithKSDK kayquinn kodacohen TheRevAl   I do not understand the mentality behind it.  Totally a worthless cause.
    Answer:non-rumor
    ###
    Tweet:furlinedPumas kodacohen people have been out there all day, every day since this started. Focus on his bravery instead.
    Answer:non-rumor
    ###
    Tweet:DreadlockDipset they said something about robbery in the beginning and the employees at that QuickTrip said they never called the police
    Answer:non-rumor
    ###
    Tweet:MotherJones this shows me that white people are slow and have no idea how to fire a gun. I also fail to see the unarmed data.
    Answer:non-rumor
    ###
    Tweet:AnonyOps Maybe the police in Ferguson are just complete trolls?
    Answer:rumor
    ###
    Tweet:AnonyOps There are A LOT of non black Americans who are VERY ANGRY at the Ferguson cops. I am a white man. Let's stand together in truth.
    Answer:rumor
    ###
    Tweet:AnonyOps Xplant So that means its ok to torch and loot someone's business?
    Answer:rumor
    ###
    Tweet:Xplant AnonyOps I'm curious how many of these protesters ever showed up to city meetings, town halls etc, to voice concerns or grievances?
    Answer:rumor
    ###
    Tweet:RT_com attacking unarmed people with automatic weapons. How brave there were!
    Answer:rumor
    ###
    Tweet:MT daphneedenis: This was a well-planned attack: the gunmen hit #CharlieHebdo in middle of weekly conference, with all staff present.
    Answer:rumor
    ###
    Tweet:CharlieHebdo's lawyer says the magazine's editor-in-chief was killed in the attack. More on this developing story: http://t.co/ryZaVLR6mM
    Answer:rumor
    ###
    Tweet:Sjcrookston AidanRoyale  nobody is forcing you to be like us. You don't have to respect us BUT BE TOLERANT OF US!!!
    Answer:rumor
    ###
    Tweet:Sjcrookston AidanRoyale  I think at your age you know that western media has got an agenda muslims
    Answer:rumor
    ###
    Tweet:Xplant AnonyOps You get 15,000 people showing up to a city meeting, that'll change minds quickly without all the needed violence &amp; mayhem
    prediction:rumor
    ###
    Tweet:"""+tweet+""" Return your answer in rumor and non rumor only.
    Answer:"""
    gen = pipeline("text-generation",model=model,tokenizer=tokenizer,temperature=0.4,max_length=1100,return_full_text=False,eos_token_id=[int(tokenizer.convert_tokens_to_ids(end_sequence))])
    #gen = pipeline("text-generation",model=model,tokenizer=tokenizer,temperature=0.4,max_length=100,return_full_text=False)

    response=gen(prompt)
    generated_text = response[0]['generated_text']
    label = generated_text.split('\n')[0]
    #print(label)
    #predictions.append(label)
    if ((label.lower()=="non rumor")or(label.lower()=="non-rumor")or(label.lower()=="non")):
        predictions.append("Non-Rumor")
    else:
        predictions.append("Rumor")